## Notebook for performing active learning to optimize anode-free lithium metal battery electrolytes

**Note: Dataset do not contain any NMC data**

## Batch: 1

In [2]:
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem import AllChem, DataStructs, PandasTools, Fragments, rdMolDescriptors, Descriptors
from rdkit.Chem.MolStandardize.rdMolStandardize import LargestFragmentChooser
# Silence non-critical RDKit warnings to minimize unnecessary outputs
from rdkit import RDLogger
lg = RDLogger.logger()
lg.setLevel(RDLogger.CRITICAL)
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.gaussian_process.kernels import RBF, ExpSineSquared, RationalQuadratic, WhiteKernel, Matern, ConstantKernel, DotProduct, PairwiseKernel 
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler
from scipy.stats import norm
from scipy.optimize import minimize
from scipy.special import erf
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
# import plotly.express as px

### Reading & standardizing datasets

In [ ]:
## in-house dataset (initial labeled dataset for training surrogate models)
df = pd.read_csv('../../datasets/in-house_label_data_wo_nmc_data.csv')
df

,solv_comb_sm,salt_comb_sm,solv_ecfp_pca_0,solv_ecfp_pca_1,solv_ecfp_pca_2,solv_ecfp_pca_3,solv_ecfp_pca_4,solv_ecfp_pca_5,solv_ecfp_pca_6,solv_ecfp_pca_7,...,norm_capacity_15,norm_capacity_16,norm_capacity_17,norm_capacity_18,norm_capacity_19,norm_capacity_20,norm_capacity_21,norm_capacity_22,norm_capacity_23,act_capacity_20
0,COCCOC,O=S(=O)(F)[N-]S(=O)(=O)F.[Li+],-0.845301,-0.995151,1.062720,-0.357552,-0.308720,0.309456,0.693325,0.613072,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,41.48700
1,COCCOC(C)C,O=S(=O)(F)[N-]S(=O)(=O)F.[Li+],-1.183255,-0.948338,0.615257,-0.582269,-1.010187,-0.522075,0.496896,0.301582,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,99.21900
2,COCCOCC(C)C,O=S(=O)(F)[N-]S(=O)(=O)F.[Li+],-1.240814,-0.970769,0.671105,-0.607789,-1.124651,-0.436617,0.628824,0.421934,...,0.021680,0.022967,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,26.75800
3,COCCOCC(C)C,O=S(=O)(F)[N-]S(=O)(=O)F.[Li+],-1.240814,-0.970769,0.671105,-0.607789,-1.124651,-0.436617,0.628824,0.421934,...,0.000460,0.000360,0.000407,0.000600,0.000593,0.000447,0.000333,0.000000,0.000000,32.54300
4,CCOCCOC(C)(C)C,O=S(=O)(F)[N-]S(=O)(=O)F.[Li+],-0.879184,-1.539457,0.507075,-0.111830,0.800175,0.133053,0.926130,-0.208395,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.92800
5,CCOCCOC(C)C,O=S(=O)(F)[N-]S(=O)(=O)F.[Li+],-1.092295,-1.519334,-0.056200,-0.093036,-0.572012,-0.583818,-0.053796,-0.111619,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,93.06700
6,CCOCCOC(C)C,O=S(=O)(F)[N-]S(=O)(=O)F.[Li+],-1.092295,-1.519334,-0.056200,-0.093036,-0.572012,-0.583818,-0.053796,-0.111619,...,0.146987,0.135520,0.122253,0.109647,0.104447,0.098607,0.088353,0.000000,0.000000,96.55700
7,CCCCOCCOC,O=S(=O)(F)[N-]S(=O)(=O)F.[Li+],-1.067892,-1.253529,0.602357,0.775134,-0.714364,1.295648,0.646532,0.057886,...,0.023487,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,42.75300
8,COCCOC1CCCC1,O=S(=O)(F)[N-]S(=O)(=O)F.[Li+],0.394637,-0.462672,0.854416,-0.416162,-1.007141,1.060702,0.662827,-0.394235,...,0.019667,0.019633,0.021167,0.022867,0.026233,0.025507,0.025267,0.000000,0.000000,69.89100
9,CCCCOCCOC(C)C,O=S(=O)(F)[N-]S(=O)(=O)F.[Li+],-1.075803,-1.758840,-0.316575,0.667843,-0.912376,0.097061,-0.035434,-0.602637,...,0.044827,0.041993,0.037160,0.043613,0.046513,0.048480,0.050413,0.000000,0.000000,84.90000


In [3]:
X = df.iloc[:,2:27] ## PCA-reduced solvent & salt descriptors and other non-molecular features
y = df['norm_capacity_3'] # normalized discharge capacity at 20th cycle (target variable)
std_scale = StandardScaler().fit(X)
X_std = std_scale.transform(X)
X_std = pd.DataFrame(X_std, columns=X.columns)

### Active learning workflow

#### Choose best hyperparameters for each kernel

In [4]:
def negative_log_likelihood_rbf(params):
    noise_level, length_scale, alpha = params
    kernel = RBF(length_scale=length_scale)
    white_kernel = WhiteKernel(noise_level=noise_level)
    composite_kernel = kernel + white_kernel
    gpr = GaussianProcessRegressor(kernel=composite_kernel, alpha=alpha, optimizer="fmin_l_bfgs_b", n_restarts_optimizer=10, random_state=42) 
    gpr.fit(X_std, y)
    pred_mean, pred_std = gpr.predict(X_std, return_std=True)
    log_likelihood = np.sum(norm.logpdf(y, loc=pred_mean, scale=pred_std))
    return -log_likelihood

def negative_log_likelihood_rq(params):
    noise_level, length_scale, alpha_k, alpha = params 
    kernel = RationalQuadratic(length_scale=length_scale, alpha=alpha_k)
    white_kernel = WhiteKernel(noise_level=noise_level)
    composite_kernel = kernel + white_kernel
    gpr = GaussianProcessRegressor(kernel=composite_kernel, alpha=alpha, optimizer="fmin_l_bfgs_b", n_restarts_optimizer=10, random_state=42)
    gpr.fit(X_std, y)
    pred_mean, pred_std = gpr.predict(X_std, return_std=True)
    log_likelihood = np.sum(norm.logpdf(y, loc=pred_mean, scale=pred_std))
    return -log_likelihood

def negative_log_likelihood_rbf_expsin(params):
    noise_level, length_scale, periodicity, alpha = params 
    kernel = RBF(length_scale=length_scale) + ExpSineSquared(length_scale=length_scale, periodicity=periodicity)
    white_kernel = WhiteKernel(noise_level=noise_level)
    composite_kernel = kernel + white_kernel
    gpr = GaussianProcessRegressor(kernel=composite_kernel, alpha=alpha, optimizer="fmin_l_bfgs_b", n_restarts_optimizer=10, random_state=42) 
    gpr.fit(X_std, y)
    pred_mean, pred_std = gpr.predict(X_std, return_std=True)
    log_likelihood = np.sum(norm.logpdf(y, loc=pred_mean, scale=pred_std))
    return -log_likelihood

def negative_log_likelihood_matern(params):
    noise_level, length_scale, alpha = params
    kernel = Matern(length_scale=length_scale, nu=1.5)
    white_kernel = WhiteKernel(noise_level=noise_level)
    composite_kernel = kernel + white_kernel
    gpr = GaussianProcessRegressor(kernel=composite_kernel, alpha=alpha, optimizer="fmin_l_bfgs_b", n_restarts_optimizer=10, random_state=42)
    gpr.fit(X_std, y)
    pred_mean, pred_std = gpr.predict(X_std, return_std=True)
    log_likelihood = np.sum(norm.logpdf(y, loc=pred_mean, scale=pred_std))
    return -log_likelihood

def negative_log_likelihood_pairwise(params):
    noise_level, length_scale, alpha = params
    kernel = PairwiseKernel(metric="polynomial")
    white_kernel = WhiteKernel(noise_level=noise_level)
    composite_kernel = kernel + white_kernel
    gpr = GaussianProcessRegressor(kernel=composite_kernel, alpha=alpha, optimizer="fmin_l_bfgs_b", n_restarts_optimizer=10, random_state=42)
    gpr.fit(X_std, y)
    pred_mean, pred_std = gpr.predict(X_std, return_std=True)
    log_likelihood = np.sum(norm.logpdf(y, loc=pred_mean, scale=pred_std))
    return -log_likelihood

##### Pairwise kernel

In [5]:
initial_guess = [0.15, 0.01, 0.02] # initial guess for noise_level, length_scale, alpha
param_bounds = [(1e-4, 1.0), (1e-5, 50.0), (1e-4, 0.1)] # bounds for noise_level, length_scale, alpha
result = minimize(negative_log_likelihood_pairwise, initial_guess, bounds=param_bounds)
optimized_hyperparameters = result.x
optimized_noise_level, optimized_length_scale, optimized_alpha = optimized_hyperparameters

## enter theses optimized hyperparameters into the combined GP model manually
print("Optimized noise_level:", optimized_noise_level)
print("Optimized length_scale:", optimized_length_scale)
print("Optimized alpha:", optimized_alpha)

/Users/riteshk/codes/miniconda3/envs/mdanalysis/lib/python3.8/site-packages/sklearn/gaussian_process/_gpr.py:659: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)
/Users/riteshk/codes/miniconda3/envs/mdanalysis/lib/python3.8/site-packages/sklearn/gaussian_process/_gpr.py:659: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)
/Users/riteshk/codes/miniconda3/envs/mdanalysis/lib/python3.8/site-packages/sklearn/gaussian_process/_gpr.py:659: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of i

Optimized noise_level: 0.15
Optimized length_scale: 0.01
Optimized alpha: 0.01997746706523124


##### RationalQuadratic kernel

In [6]:
initial_guess = [0.15, 0.01, 0.01, 0.02] # initial guess for noise_level, length_scale, alpha_k, alpha
param_bounds = [(1e-4, 1.0), (1e-5, 50.0), (1e-5, 50.0), (1e-4, 0.1)] # bounds for noise_level, length_scale, alpha_k, alpha
result = minimize(negative_log_likelihood_rq, initial_guess, bounds=param_bounds)
optimized_hyperparameters = result.x
optimized_noise_level, optimized_length_scale, optimized_alpha_k, optimized_alpha = optimized_hyperparameters

## enter theses optimized hyperparameters into the combined GP model manually
print("Optimized noise_level:", optimized_noise_level)
print("Optimized length_scale:", optimized_length_scale)
print("Optimized alpha_k:", optimized_alpha_k)
print("Optimized alpha:", optimized_alpha)

/Users/riteshk/codes/miniconda3/envs/mdanalysis/lib/python3.8/site-packages/sklearn/gaussian_process/kernels.py:419: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/riteshk/codes/miniconda3/envs/mdanalysis/lib/python3.8/site-packages/sklearn/gaussian_process/kernels.py:419: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/riteshk/codes/miniconda3/envs/mdanalysis/lib/python3.8/site-packages/sklearn/gaussian_process/kernels.py:419: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warning

Optimized noise_level: 0.15000000059190507
Optimized length_scale: 0.010000015109394703
Optimized alpha_k: 0.009999987904032791
Optimized alpha: 0.006981088333228721


##### Matern-3/2 kernel

In [7]:
initial_guess = [0.15, 0.01, 0.02] # initial guess for noise_level, length_scale, alpha
param_bounds = [(1e-4, 1.0), (1e-5, 50.0), (1e-4, 0.1)] # bounds for noise_level, length_scale, alpha
result = minimize(negative_log_likelihood_matern, initial_guess, bounds=param_bounds)
optimized_hyperparameters = result.x
optimized_noise_level, optimized_length_scale, optimized_alpha = optimized_hyperparameters

## enter theses optimized hyperparameters into the combined GP model manually
print("Optimized noise_level:", optimized_noise_level)
print("Optimized length_scale:", optimized_length_scale)
print("Optimized alpha:", optimized_alpha)

/Users/riteshk/codes/miniconda3/envs/mdanalysis/lib/python3.8/site-packages/sklearn/gaussian_process/kernels.py:419: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/riteshk/codes/miniconda3/envs/mdanalysis/lib/python3.8/site-packages/sklearn/gaussian_process/kernels.py:419: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/riteshk/codes/miniconda3/envs/mdanalysis/lib/python3.8/site-packages/sklearn/gaussian_process/kernels.py:419: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warning

Optimized noise_level: 0.15
Optimized length_scale: 0.01
Optimized alpha: 0.007805939588338199


##### RBF-ExpineSquared kernel

In [8]:
initial_guess = [0.15, 0.01, 1.0, 0.02] # initial guess for noise_level, length_scale, periodicity, alpha
param_bounds = [(1e-4, 1.0), (1e-5, 50.0), (1e-2, 10.0), (1e-4, 0.1)] # bounds for noise_level, length_scale, periodicity, alpha
result = minimize(negative_log_likelihood_rbf_expsin, initial_guess, bounds=param_bounds)
optimized_hyperparameters = result.x
optimized_noise_level, optimized_length_scale, optimized_periodicity, optimized_alpha = optimized_hyperparameters

## enter theses optimized hyperparameters into the combined GP model manually
print("Optimized noise_level:", optimized_noise_level)
print("Optimized length_scale:", optimized_length_scale)
print("Optimized periodicity:", optimized_periodicity)
print("Optimized alpha:", optimized_alpha)

/Users/riteshk/codes/miniconda3/envs/mdanalysis/lib/python3.8/site-packages/sklearn/gaussian_process/kernels.py:419: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/riteshk/codes/miniconda3/envs/mdanalysis/lib/python3.8/site-packages/sklearn/gaussian_process/kernels.py:419: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/riteshk/codes/miniconda3/envs/mdanalysis/lib/python3.8/site-packages/sklearn/gaussian_process/kernels.py:419: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warning

Optimized noise_level: 0.15
Optimized length_scale: 0.01
Optimized periodicity: 1.0
Optimized alpha: 0.020007821450177304


/Users/riteshk/codes/miniconda3/envs/mdanalysis/lib/python3.8/site-packages/sklearn/gaussian_process/kernels.py:419: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(


#### Train surrogate models

Note: no need to run again, saved model checkpoints have been provided

In [ ]:
## add optimized hyperparameters from previous step
optimized_pairwise_kernel = PairwiseKernel(metric="polynomial") + WhiteKernel(noise_level=0.15)
optimized_matern_kernel = Matern(length_scale=0.01, nu=1.5) + WhiteKernel(noise_level=0.15)
optimized_rbfexpsin_kernel = RBF(length_scale=0.01) + ExpSineSquared(length_scale=0.01, periodicity=1.0) + WhiteKernel(noise_level=0.15)
optimized_rq_kernel = RationalQuadratic(length_scale=0.010000015109394703, alpha=0.009999987904032791) + WhiteKernel(noise_level=0.15000000059190507)

gpr_models = [GaussianProcessRegressor(kernel=optimized_pairwise_kernel, alpha=0.01997746706523124, optimizer="fmin_l_bfgs_b", n_restarts_optimizer=10, random_state=42),
              GaussianProcessRegressor(kernel=optimized_matern_kernel, alpha=0.007805939588338199, optimizer="fmin_l_bfgs_b", n_restarts_optimizer=10, random_state=42),
              GaussianProcessRegressor(kernel=optimized_rq_kernel, alpha=0.006981088333228721, optimizer="fmin_l_bfgs_b", n_restarts_optimizer=10, random_state=42),
              GaussianProcessRegressor(kernel=optimized_rbfexpsin_kernel, alpha=0.020007821450177304, optimizer="fmin_l_bfgs_b", n_restarts_optimizer=10, random_state=42)]

model_names = ['../../models/batch-1/pairwise_batch1_wo_nmc_data.pkl', '../../models/batch-1/matern_batch1_wo_nmc_data.pkl', '../../models/batch-1/rq_batch1_wo_nmc_data.pkl', '../../models/batch-1/rbf-ess_batch1_wo_nmc_data.pkl']
k = 0
for model in gpr_models:
    print("fitting model: ", k)
    model.fit(X_std, y)
    pickle.dump(model, open(model_names[k], 'wb'))
    k += 1

fitting model:  0
fitting model:  1
fitting model:  2
fitting model:  3


/Users/riteshk/codes/miniconda3/envs/mdanalysis/lib/python3.8/site-packages/sklearn/gaussian_process/kernels.py:419: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(


#### BMA: aggregate predictions 

##### Acquisition function (Expected improvement)

In [12]:
## final corrected & verified one to be used
def calc_EI(y_pred, y_pred_un, y_pred_un_uncer, epsilon=0.01):
    y_best = np.max(y_pred)
    EI = []
    explore = []
    exploit = []

    for i in range(len(y_pred_un)):
        if y_pred_un_uncer[i] != 0:
            
            # Calculate the cumulative distribution function (CDF) for the Gaussian distribution
            z = (y_pred_un[i] - y_best - epsilon) / y_pred_un_uncer[i]
            # z = (y_pred_un[i] - y_best - epsilon) / y_pred_un_uncer[i]
            cdf_z = 0.5 * (1 + erf(z / np.sqrt(2)))
            pdf_z = np.exp(-0.5 * z**2) / np.sqrt(2 * np.pi)

            # Calculate Expected Improvement
            expected_improvement = y_pred_un_uncer[i] * (z * cdf_z) + y_pred_un_uncer[i] * pdf_z
            exploitation = y_pred_un_uncer[i] * z * cdf_z
            exploration = y_pred_un_uncer[i] * pdf_z
            EI.append(expected_improvement)
            explore.append(exploration)
            exploit.append(exploitation)
        else:
            EI.append(0.0)
    return EI, exploit, explore

In [4]:
## virtual search space (initial containing 1 million electrolytes)
path = '/Users/riteshk/Library/CloudStorage/Box-Box/Research-postdoc/AD-AFB/data-codes-sharing/datasets'
df_unlabel = pd.read_csv('../../datasets/virtual_search_space_1million.csv') ## download from Box and keep in the same directory
df_unlabel

,solv_comb_sm,salt_comb_sm,solv_ecfp_pca_0,solv_ecfp_pca_1,solv_ecfp_pca_2,solv_ecfp_pca_3,solv_ecfp_pca_4,solv_ecfp_pca_5,solv_ecfp_pca_6,solv_ecfp_pca_7,...,salt_ecfp_pca_5,salt_ecfp_pca_6,salt_ecfp_pca_7,salt_ecfp_pca_8,salt_ecfp_pca_9,mol_wt_solv,mol_wt_salt,conc_salt_1,theor_capacity,amt_electrolyte
0,CN(C)C=O,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,-0.325380,-0.919052,-0.279367,-0.373535,0.936724,-0.161937,-0.377185,0.259444,...,0.128733,-0.322339,0.258767,0.301779,-0.272648,73.052764,186.939685,1.0,150,50
1,CN1CCN(C)C1=O,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,1.013958,-0.868963,0.700464,0.728341,0.681048,-1.181697,0.018457,0.793475,...,0.128733,-0.322339,0.258767,0.301779,-0.272648,114.079313,186.939685,1.0,150,50
2,CN(C)C(=O)N(C)C,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,-0.571448,0.079720,-0.117841,-0.342583,1.301206,-0.264839,-0.516489,0.217482,...,0.128733,-0.322339,0.258767,0.301779,-0.272648,116.094963,186.939685,1.0,150,50
3,CB(C)C=O,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,-0.152976,-1.321495,0.631553,-0.113769,0.728616,-0.486733,-0.311699,0.151570,...,0.128733,-0.322339,0.258767,0.301779,-0.272648,70.058995,186.939685,1.0,150,50
4,[CH2]N(C)C=O,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,-0.283297,-0.876057,-0.318283,-0.547668,0.955202,-0.313588,-0.356929,0.299023,...,0.128733,-0.322339,0.258767,0.301779,-0.272648,72.044939,186.939685,1.0,150,50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999994,CC1ON(C)C(C)C1S(C)(=O)=O,[Li+].O=C1O[B-](F)(F)OC1=O,0.771012,-1.078565,0.565077,0.216605,0.613562,-0.742397,-0.257837,0.243970,...,-0.387958,-0.076666,0.008496,0.161338,0.109063,193.077264,144.001775,1.0,150,50
999995,COC(=O)C1(N2CCCN(C)C2=O)CCCC1,[Li+].O=C1O[B-](F)(F)OC1=O,0.881747,0.625623,1.808971,0.306581,0.123581,-0.469934,-0.451538,1.337094,...,-0.387958,-0.076666,0.008496,0.161338,0.109063,240.147392,144.001775,1.0,150,50
999996,COC(=O)N1CCC(OS(C)(=O)=O)CC1=O,[Li+].O=C1O[B-](F)(F)OC1=O,0.700964,0.588889,1.459777,0.806459,-0.042994,-0.081386,0.196396,-0.195416,...,-0.387958,-0.076666,0.008496,0.161338,0.109063,251.046358,144.001775,1.0,150,50
999997,CN(C)C(=O)CN(C)C(=O)C1CCCC1(F)F,[Li+].O=C1O[B-](F)(F)OC1=O,0.558235,0.889818,-0.240008,-0.734436,0.749283,0.799906,-0.856418,0.214675,...,-0.387958,-0.076666,0.008496,0.161338,0.109063,248.133634,144.001775,1.0,150,50


In [14]:
X_un = df_unlabel.iloc[:,2:]
X_un_std = std_scale.transform(X_un)
X_un_std = pd.DataFrame(X_un_std, columns=X_un.columns)

##### Calculate model weights & obtained aggregated mean ($\mu^{aggr}$), uncertainty ($\sigma^{aggr}$), & EI ($EI^{aggr}$)

In [ ]:
# Calculate model weights using BMA (first order)
model_names = ['../../models/batch-1/pairwise_batch1_wo_nmc_data.pkl', '../../models/batch-1/matern_batch1_wo_nmc_data.pkl', '../../models/batch-1/rq_batch1_wo_nmc_data.pkl', '../../models/batch-1/rbf-ess_batch1_wo_nmc_data.pkl']
model_weights = []
uncertainties = []
predictions = []
y_label_preds = []
for model in model_names:
    gpr = pickle.load(open(model, 'rb'))
    y_un = gpr.predict(X_un_std)
    predictions.append(y_un)
    individual_uncertainties = gpr.predict(X_un_std, return_std=True)[1]
    uncertainties.append(individual_uncertainties)
    likelihoods = norm.pdf(y_un, loc=gpr.predict(X_un_std), scale=individual_uncertainties)
    prior_beliefs = 1.0  # Non-informative prior
    posterior = likelihoods * prior_beliefs
    model_weights.append(posterior / np.sum(posterior))
    y_ = gpr.predict(X_std)
    y_label_preds.append(y_)

In [16]:
df_unlabel['uncertainty_1'] = uncertainties[0]; df_unlabel['uncertainty_2'] = uncertainties[1]; df_unlabel['uncertainty_3'] = uncertainties[2]; df_unlabel['uncertainty_4'] = uncertainties[3]
df_unlabel['prediction_1'] = predictions[0]; df_unlabel['prediction_2'] = predictions[1]; df_unlabel['prediction_3'] = predictions[2]; df_unlabel['prediction_4'] = predictions[3]
df_unlabel['explore_1'] = calc_EI(y_label_preds[0], predictions[0], uncertainties[0])[2]; df_unlabel['exploit_1'] = calc_EI(y_label_preds[0], predictions[0], uncertainties[0])[1]
df_unlabel['explore_2'] = calc_EI(y_label_preds[1], predictions[1], uncertainties[1])[2]; df_unlabel['exploit_2'] = calc_EI(y_label_preds[1], predictions[1], uncertainties[1])[1]
df_unlabel['explore_3'] = calc_EI(y_label_preds[2], predictions[2], uncertainties[2])[2]; df_unlabel['exploit_3'] = calc_EI(y_label_preds[2], predictions[2], uncertainties[2])[1]
df_unlabel['explore_4'] = calc_EI(y_label_preds[3], predictions[3], uncertainties[3])[2]; df_unlabel['exploit_4'] = calc_EI(y_label_preds[3], predictions[3], uncertainties[3])[1]
df_unlabel['EI_1'] = calc_EI(y_label_preds[0], predictions[0], uncertainties[0])[0]; df_unlabel['EI_2'] = calc_EI(y_label_preds[1], predictions[1], uncertainties[1])[0]; df_unlabel['EI_3'] = calc_EI(y_label_preds[2], predictions[2], uncertainties[2])[0]; df_unlabel['EI_4'] = calc_EI(y_label_preds[3], predictions[3], uncertainties[3])[0]
df_unlabel

,solv_comb_sm,salt_comb_sm,solv_ecfp_pca_0,solv_ecfp_pca_1,solv_ecfp_pca_2,solv_ecfp_pca_3,solv_ecfp_pca_4,solv_ecfp_pca_5,solv_ecfp_pca_6,solv_ecfp_pca_7,...,explore_2,exploit_2,explore_3,exploit_3,explore_4,exploit_4,EI_1,EI_2,EI_3,EI_4
0,CN(C)C=O,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,-0.325380,-0.919052,-0.279367,-0.373535,0.936724,-0.161937,-0.377185,0.259444,...,0.141100,-0.076228,0.049084,-0.035963,0.153775,-0.080438,0.009104,0.064872,0.013121,0.073336
1,CN1CCN(C)C1=O,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,1.013958,-0.868963,0.700464,0.728341,0.681048,-1.181697,0.018457,0.793475,...,0.208707,-0.115061,0.051355,-0.037783,0.261144,-0.127255,0.014627,0.093646,0.013572,0.133889
2,CN(C)C(=O)N(C)C,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,-0.571448,0.079720,-0.117841,-0.342583,1.301206,-0.264839,-0.516489,0.217482,...,0.108119,-0.077398,0.045715,-0.034260,0.120698,-0.086205,0.005997,0.030721,0.011456,0.034493
3,CB(C)C=O,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,-0.152976,-1.321495,0.631553,-0.113769,0.728616,-0.486733,-0.311699,0.151570,...,0.063256,-0.045156,0.042263,-0.031839,0.045476,-0.034461,0.005828,0.018100,0.010424,0.011015
4,[CH2]N(C)C=O,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,-0.283297,-0.876057,-0.318283,-0.547668,0.955202,-0.313588,-0.356929,0.299023,...,0.156044,-0.082452,0.050242,-0.036699,0.171060,-0.088728,0.010978,0.073592,0.013543,0.082331
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999994,CC1ON(C)C(C)C1S(C)(=O)=O,[Li+].O=C1O[B-](F)(F)OC1=O,0.771012,-1.078565,0.565077,0.216605,0.613562,-0.742397,-0.257837,0.243970,...,0.294607,-0.169932,0.087822,-0.058450,0.388402,-0.173168,0.687822,0.124675,0.029372,0.215234
999995,COC(=O)C1(N2CCCN(C)C2=O)CCCC1,[Li+].O=C1O[B-](F)(F)OC1=O,0.881747,0.625623,1.808971,0.306581,0.123581,-0.469934,-0.451538,1.337094,...,0.294658,-0.169935,0.088574,-0.058775,0.388399,-0.173169,0.898729,0.124723,0.029799,0.215230
999996,COC(=O)N1CCC(OS(C)(=O)=O)CC1=O,[Li+].O=C1O[B-](F)(F)OC1=O,0.700964,0.588889,1.459777,0.806459,-0.042994,-0.081386,0.196396,-0.195416,...,0.294586,-0.169931,0.087857,-0.058474,0.388387,-0.173178,0.698650,0.124655,0.029383,0.215209
999997,CN(C)C(=O)CN(C)C(=O)C1CCCC1(F)F,[Li+].O=C1O[B-](F)(F)OC1=O,0.558235,0.889818,-0.240008,-0.734436,0.749283,0.799906,-0.856418,0.214675,...,0.294642,-0.169934,0.088339,-0.058668,0.388403,-0.173167,0.801514,0.124708,0.029671,0.215235


In [17]:
def calc_aggr_uncer(uncer_1, w_1, pred_1, uncer_2, w_2, pred_2, uncer_3, w_3, pred_3, uncer_4, w_4, pred_4):
    uncer = [uncer_1, uncer_2, uncer_3, uncer_4]
    pred = [pred_1, pred_2, pred_3, pred_4]
    weight = [w_1, w_2, w_3, w_4]
    pred_aggr = w_1 * pred_1 + w_2 * pred_2 + w_3 * pred_3 + w_4 * pred_4
    sum = 0
    for i in range(4):
        sum += weight[i] * (uncer[i]**2 + (pred[i] - pred_aggr)**2)
    aggr_uncer = np.sqrt(sum)
    return aggr_uncer

In [18]:
df_unlabel['prediction_aggr'] = df_unlabel['prediction_1'] * model_weights[0] + df_unlabel['prediction_2'] * model_weights[1] + df_unlabel['prediction_3'] * model_weights[2] + df_unlabel['prediction_4'] * model_weights[3]
df_unlabel['uncertainty_aggr'] = calc_aggr_uncer(df_unlabel['uncertainty_1'], model_weights[0], df_unlabel['prediction_1'], df_unlabel['uncertainty_2'], model_weights[1], df_unlabel['prediction_2'], df_unlabel['uncertainty_3'], model_weights[2], df_unlabel['prediction_3'], df_unlabel['uncertainty_4'], model_weights[3], df_unlabel['prediction_4'])
df_unlabel['explore_aggr'] = df_unlabel['explore_1'] * model_weights[0] + df_unlabel['explore_2'] * model_weights[1] + df_unlabel['explore_3'] * model_weights[2] + df_unlabel['explore_4'] * model_weights[3]
df_unlabel['exploit_aggr'] = df_unlabel['exploit_1'] * model_weights[0] + df_unlabel['exploit_2'] * model_weights[1] + df_unlabel['exploit_3'] * model_weights[2] + df_unlabel['exploit_4'] * model_weights[3]
df_unlabel['ratio_aggr'] = df_unlabel['exploit_aggr'] / df_unlabel['explore_aggr']

## 'EI_aggr' is the final rank by which candidate electrolytes are selected for experimental validation
df_unlabel['EI_aggr'] = df_unlabel['EI_1'] * model_weights[0] + df_unlabel['EI_2'] * model_weights[1] + df_unlabel['EI_3'] * model_weights[2] + df_unlabel['EI_4'] * model_weights[3]
df_unlabel

,solv_comb_sm,salt_comb_sm,solv_ecfp_pca_0,solv_ecfp_pca_1,solv_ecfp_pca_2,solv_ecfp_pca_3,solv_ecfp_pca_4,solv_ecfp_pca_5,solv_ecfp_pca_6,solv_ecfp_pca_7,...,EI_1,EI_2,EI_3,EI_4,prediction_aggr,uncertainty_aggr,explore_aggr,exploit_aggr,ratio_aggr,EI_aggr
0,CN(C)C=O,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,-0.325380,-0.919052,-0.279367,-0.373535,0.936724,-0.161937,-0.377185,0.259444,...,0.009104,0.064872,0.013121,0.073336,3.021278e-06,0.001508,6.899606e-07,-4.016823e-07,-0.582181,2.882784e-07
1,CN1CCN(C)C1=O,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,1.013958,-0.868963,0.700464,0.728341,0.681048,-1.181697,0.018457,0.793475,...,0.014627,0.093646,0.013572,0.133889,1.723467e-06,0.001432,7.153395e-07,-4.082144e-07,-0.570658,3.071251e-07
2,CN(C)C(=O)N(C)C,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,-0.571448,0.079720,-0.117841,-0.342583,1.301206,-0.264839,-0.516489,0.217482,...,0.005997,0.030721,0.011456,0.034493,1.111414e-06,0.001248,4.435818e-07,-3.248110e-07,-0.732246,1.187707e-07
3,CB(C)C=O,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,-0.152976,-1.321495,0.631553,-0.113769,0.728616,-0.486733,-0.311699,0.151570,...,0.005828,0.018100,0.010424,0.011015,3.215670e-06,0.001375,4.122597e-07,-3.076336e-07,-0.746213,1.046261e-07
4,[CH2]N(C)C=O,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,-0.283297,-0.876057,-0.318283,-0.547668,0.955202,-0.313588,-0.356929,0.299023,...,0.010978,0.073592,0.013543,0.082331,2.809687e-06,0.001494,7.080695e-07,-4.078159e-07,-0.575955,3.002537e-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999994,CC1ON(C)C(C)C1S(C)(=O)=O,[Li+].O=C1O[B-](F)(F)OC1=O,0.771012,-1.078565,0.565077,0.216605,0.613562,-0.742397,-0.257837,0.243970,...,0.687822,0.124675,0.029372,0.215234,5.162658e-07,0.001921,8.825805e-07,-3.919648e-07,-0.444112,4.906157e-07
999995,COC(=O)C1(N2CCCN(C)C2=O)CCCC1,[Li+].O=C1O[B-](F)(F)OC1=O,0.881747,0.625623,1.808971,0.306581,0.123581,-0.469934,-0.451538,1.337094,...,0.898729,0.124723,0.029799,0.215230,6.350017e-07,0.001950,8.908653e-07,-3.463583e-07,-0.388789,5.445070e-07
999996,COC(=O)N1CCC(OS(C)(=O)=O)CC1=O,[Li+].O=C1O[B-](F)(F)OC1=O,0.700964,0.588889,1.459777,0.806459,-0.042994,-0.081386,0.196396,-0.195416,...,0.698650,0.124655,0.029383,0.215209,5.174795e-07,0.001925,8.828657e-07,-3.909505e-07,-0.442820,4.919151e-07
999997,CN(C)C(=O)CN(C)C(=O)C1CCCC1(F)F,[Li+].O=C1O[B-](F)(F)OC1=O,0.558235,0.889818,-0.240008,-0.734436,0.749283,0.799906,-0.856418,0.214675,...,0.801514,0.124708,0.029671,0.215235,5.828793e-07,0.001935,8.883419e-07,-3.685728e-07,-0.414900,5.197691e-07


##### Save top 5000 predictions

In [19]:
df_unlabel_ = df_unlabel.copy()
df_unlabel_ = df_unlabel_.sort_values(by='EI_aggr', ascending=False)
df_unlabel_5000 = df_unlabel_.iloc[:5000,:]
df_unlabel_5000

,solv_comb_sm,salt_comb_sm,solv_ecfp_pca_0,solv_ecfp_pca_1,solv_ecfp_pca_2,solv_ecfp_pca_3,solv_ecfp_pca_4,solv_ecfp_pca_5,solv_ecfp_pca_6,solv_ecfp_pca_7,...,EI_1,EI_2,EI_3,EI_4,prediction_aggr,uncertainty_aggr,explore_aggr,exploit_aggr,ratio_aggr,EI_aggr
15691,CCBCC,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,-0.276040,-1.693605,0.055654,0.173017,0.514091,-0.218305,-0.279889,-0.072427,...,0.013992,0.189308,0.018074,0.246113,0.000008,0.002711,7.258115e-07,7.483006e-07,1.030985,1.474112e-06
50,CCB(C)C,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,-0.326952,-1.721520,0.117360,0.189181,0.553021,-0.289895,-0.275382,0.018632,...,0.014015,0.176516,0.018048,0.221232,0.000008,0.002737,7.367775e-07,7.014297e-07,0.952024,1.438207e-06
11890,[CH2]C[CH]CC,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,-0.289406,-1.730150,0.121586,0.189316,0.521206,-0.270810,-0.305220,-0.060533,...,0.011583,0.170072,0.017290,0.215895,0.000007,0.002600,7.594389e-07,5.224617e-07,0.687958,1.281901e-06
1306,CCB(C)CC,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,-0.241874,-1.659053,0.070966,0.121841,0.534775,-0.199106,-0.246681,-0.196227,...,0.017992,0.156921,0.018083,0.210494,0.000008,0.002635,8.023972e-07,4.789572e-07,0.596908,1.281354e-06
2592,CCB(F)F,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,-0.339034,-1.762689,0.143557,0.141353,0.632132,-0.325214,-0.211159,-0.059723,...,0.015519,0.148580,0.017947,0.179678,0.000008,0.002708,7.914859e-07,4.828198e-07,0.610017,1.274306e-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59874,CCCOC[SiH2]C,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,-0.748142,-1.769569,0.309586,0.382603,0.024257,0.252883,0.085163,0.037783,...,0.012995,0.087852,0.016430,0.065836,0.000008,0.002395,8.634022e-07,-2.235243e-07,-0.258888,6.398779e-07
88642,[CH]CC(C)C,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,-0.513242,-1.543712,-0.112536,-0.390204,-0.051504,-1.177768,-0.369502,-0.090587,...,0.023481,0.132982,0.019244,0.112343,0.000005,0.002014,9.107242e-07,-2.708482e-07,-0.297399,6.398761e-07
184317,CC1CCC(C)(Cl)C1,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,1.305924,-1.380335,0.599251,-0.690555,0.297120,0.166016,-0.989821,-0.020022,...,0.034917,0.192563,0.015375,0.271085,0.000003,0.001760,9.115458e-07,-2.716783e-07,-0.298041,6.398675e-07
324629,CCC(C)N1CCC(=O)N(C)C2(CCCC2)C1=O,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,1.068452,-0.768314,0.038743,0.558979,-0.241913,-1.526501,-0.851348,0.953707,...,0.134864,0.196593,0.020543,0.262330,0.000002,0.001737,9.592667e-07,-3.194842e-07,-0.333050,6.397825e-07


In [20]:
df_unlabel_uniq = df_unlabel_5000.drop_duplicates(subset=['solv_comb_sm'], keep='first') ## only keeping unique solvent combinations for selection purposes; these compounds were manually searched in emolecules to finf purchasable compounds
df_unlabel_uniq[['solv_comb_sm', 'salt_comb_sm', 'prediction_aggr', 'uncertainty_aggr', 'explore_aggr', 'exploit_aggr', 'ratio_aggr', 'EI_aggr']].to_csv('../datasets/batch-1/top_5000_suggestions_batch1_uniq_solvents_wo_nmc_data.csv', index=False)

In [ ]:
df_unlabel_uniq = pd.read_csv('../../datasets/batch-1/top_5000_suggestions_batch1_uniq_solvents_wo_nmc_data.csv')
df_unlabel_uniq['solv_comb_sm'] = df_unlabel_uniq['solv_comb_sm'].apply(lambda x: Chem.MolToSmiles(Chem.MolFromSmiles(x)))
df_unlabel_uniq

,solv_comb_sm,salt_comb_sm,prediction_aggr,uncertainty_aggr,explore_aggr,exploit_aggr,ratio_aggr,EI_aggr
0,CCBCC,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,0.000008,0.002711,7.258115e-07,7.483006e-07,1.030985,1.474112e-06
1,CCB(C)C,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,0.000008,0.002737,7.367775e-07,7.014297e-07,0.952024,1.438207e-06
2,[CH2]C[CH]CC,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,0.000007,0.002600,7.594389e-07,5.224617e-07,0.687958,1.281901e-06
3,CCB(C)CC,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,0.000008,0.002635,8.023972e-07,4.789572e-07,0.596908,1.281354e-06
4,CCB(F)F,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,0.000008,0.002708,7.914859e-07,4.828198e-07,0.610017,1.274306e-06
...,...,...,...,...,...,...,...,...
4995,CCCOC[SiH2]C,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,0.000008,0.002395,8.634022e-07,-2.235243e-07,-0.258888,6.398779e-07
4996,[CH]CC(C)C,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,0.000005,0.002014,9.107242e-07,-2.708482e-07,-0.297399,6.398761e-07
4997,CC1CCC(C)(Cl)C1,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,0.000003,0.001760,9.115458e-07,-2.716783e-07,-0.298041,6.398675e-07
4998,CCC(C)N1CCC(=O)N(C)C2(CCCC2)C1=O,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,0.000002,0.001737,9.592667e-07,-3.194842e-07,-0.333050,6.397825e-07


In [10]:
path = '/Users/riteshk/Library/CloudStorage/Box-Box/Research-postdoc/AD-AFB/data-codes-sharing/datasets'
df_unlabel_uniq_ = pd.read_csv(f'{path}/batch-1/top_5000_suggestions_batch1_uniq_solvents.csv')
df_unlabel_uniq_

,solv_comb_sm,salt_comb_sm,prediction_aggr,uncertainty_aggr,explore_aggr,exploit_aggr,ratio_aggr,EI_aggr
0,COC1(C(F)F)CCCCC1,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,0.000005,0.002300,8.438763e-07,6.042866e-07,0.716084,1.448163e-06
1,COC1(C(F)(F)F)CCCC1,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,0.000005,0.002295,8.434955e-07,5.883782e-07,0.697547,1.431874e-06
2,COC1(C(C)(F)F)CCCCC1,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,0.000004,0.002259,8.364115e-07,5.906943e-07,0.706225,1.427106e-06
3,COC1(C(F)F)CCCC1,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,0.000005,0.002308,8.513511e-07,5.730075e-07,0.673057,1.424359e-06
4,COC1(F)CCCCC1,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,0.000005,0.002275,8.500037e-07,5.640064e-07,0.663534,1.414010e-06
...,...,...,...,...,...,...,...,...
4995,COC1(N(C)C)CCCN(C=O)C1,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,0.000003,0.001721,8.959106e-07,-2.235411e-07,-0.249513,6.723695e-07
4996,CC(C)S(=O)(=O)N1CCCC(=O)C1(C)C,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,0.000003,0.001751,8.938906e-07,-2.215512e-07,-0.247850,6.723394e-07
4997,COC(C)(C)CN1CCCC1,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,0.000004,0.001807,8.884141e-07,-2.160760e-07,-0.243215,6.723381e-07
4998,CC1(C)CC(F)(F)CCC1=O,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,0.000004,0.001818,8.932022e-07,-2.208867e-07,-0.247298,6.723154e-07


In [12]:
df_unlabel_uniq_['solv_comb_sm'] = df_unlabel_uniq_['solv_comb_sm'].apply(lambda x: Chem.MolToSmiles(Chem.MolFromSmiles(x)))
# df_sugg_comm = pd.merge(df_unlabel_uniq_, df_unlabel_uniq, on=['solv_comb_sm', 'salt_comb_sm'], how='inner')
df_sugg_comm = pd.merge(df_unlabel_uniq_, df_unlabel_uniq, on=['solv_comb_sm'], how='inner')
df_sugg_comm

,solv_comb_sm,salt_comb_sm_x,prediction_aggr_x,uncertainty_aggr_x,explore_aggr_x,exploit_aggr_x,ratio_aggr_x,EI_aggr_x,salt_comb_sm_y,prediction_aggr_y,uncertainty_aggr_y,explore_aggr_y,exploit_aggr_y,ratio_aggr_y,EI_aggr_y
0,COC1(C(F)F)CCCCC1,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,0.000005,0.002300,8.438763e-07,6.042866e-07,0.716084,1.448163e-06,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,0.000004,0.002087,9.950424e-07,1.007081e-07,0.101210,1.095750e-06
1,COC1(C(F)(F)F)CCCC1,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,0.000005,0.002295,8.434955e-07,5.883782e-07,0.697547,1.431874e-06,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,0.000004,0.002086,9.900109e-07,1.049899e-07,0.106049,1.095001e-06
2,COC1(C(C)(F)F)CCCCC1,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,0.000004,0.002259,8.364115e-07,5.906943e-07,0.706225,1.427106e-06,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,0.000003,0.002052,9.635958e-07,1.156936e-07,0.120064,1.079289e-06
3,COC1(C(F)F)CCCC1,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,0.000005,0.002308,8.513511e-07,5.730075e-07,0.673057,1.424359e-06,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,0.000004,0.002120,9.997478e-07,1.188989e-07,0.118929,1.118647e-06
4,COC1(F)CCCCC1,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,0.000005,0.002275,8.500037e-07,5.640064e-07,0.663534,1.414010e-06,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,0.000004,0.002074,9.960621e-07,8.323341e-08,0.083562,1.079296e-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3549,CC1(C=O)CCSC1,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,0.000004,0.001853,8.904616e-07,-2.175327e-07,-0.244292,6.729289e-07,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,0.000003,0.001842,9.422643e-07,-2.315955e-07,-0.245786,7.106688e-07
3550,CCC1N(C)CCCC1(C)C,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,0.000003,0.001708,8.933947e-07,-2.206972e-07,-0.247032,6.726976e-07,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,0.000002,0.001765,9.660388e-07,-2.919582e-07,-0.302222,6.740806e-07
3551,CCOCC1(C)CCN(C)CC1,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,0.000003,0.001709,8.956232e-07,-2.229611e-07,-0.248945,6.726621e-07,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,0.000002,0.001751,9.635770e-07,-3.092890e-07,-0.320980,6.542880e-07
3552,CC1(C)CCS(=O)C1,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,0.000004,0.001848,8.924772e-07,-2.198683e-07,-0.246357,6.726090e-07,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,0.000003,0.001853,9.467636e-07,-2.228336e-07,-0.235364,7.239300e-07


### Check how many common solvents in fist batch of labeled dataset

In [ ]:
df_label_all = pd.read_csv('../../datasets/label_all_ecfp_pca_add_feat_incl_b7_090824.csv')
df_label_all

,solv_comb_sm,salt_comb_sm,batch,solv_ecfp_pca_0,solv_ecfp_pca_1,solv_ecfp_pca_2,solv_ecfp_pca_3,solv_ecfp_pca_4,solv_ecfp_pca_5,solv_ecfp_pca_6,...,norm_capacity_15,norm_capacity_16,norm_capacity_17,norm_capacity_18,norm_capacity_19,norm_capacity_20,norm_capacity_21,norm_capacity_22,norm_capacity_23,expt_test
0,COCCOC,O=S(=O)(F)[N-]S(=O)(=O)F.[Li+],0.0,-0.845301,-0.995151,1.062720,-0.357552,-0.308720,0.309456,0.693325,...,0.00000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.0,0.0,0.0
1,COCCOC(C)C,O=S(=O)(F)[N-]S(=O)(=O)F.[Li+],0.0,-1.183255,-0.948338,0.615257,-0.582269,-1.010187,-0.522075,0.496896,...,0.00000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.0,0.0,0.0
2,COCCOCC(C)C,O=S(=O)(F)[N-]S(=O)(=O)F.[Li+],0.0,-1.240814,-0.970769,0.671105,-0.607789,-1.124651,-0.436617,0.628824,...,0.02168,0.022967,0.000000,0.0000,0.000000,0.000000,0.000000,0.0,0.0,0.0
3,COCCOCC(C)C,O=S(=O)(F)[N-]S(=O)(=O)F.[Li+],0.0,-1.240814,-0.970769,0.671105,-0.607789,-1.124651,-0.436617,0.628824,...,0.00046,0.000360,0.000407,0.0006,0.000593,0.000447,0.000333,0.0,0.0,0.0
4,CCOCCOC(C)(C)C,O=S(=O)(F)[N-]S(=O)(=O)F.[Li+],0.0,-0.879184,-1.539457,0.507075,-0.111830,0.800175,0.133053,0.926130,...,0.00000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203,CCOCCCCOC,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0
204,COCCCCOCC(F)(F)F,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0
205,COCCCOCC(C)(C)C,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0
206,COCCCOCC(F)(F)C(F)F,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0


In [15]:
df_b1 = df_label_all.loc[df_label_all['expt_test'] == 1]
df_b1

,solv_comb_sm,salt_comb_sm,batch,solv_ecfp_pca_0,solv_ecfp_pca_1,solv_ecfp_pca_2,solv_ecfp_pca_3,solv_ecfp_pca_4,solv_ecfp_pca_5,solv_ecfp_pca_6,...,norm_capacity_15,norm_capacity_16,norm_capacity_17,norm_capacity_18,norm_capacity_19,norm_capacity_20,norm_capacity_21,norm_capacity_22,norm_capacity_23,expt_test
58,CN1C(C)(C)CCCC1(C)C,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,1.0,1.154581,-1.168030,0.649902,-0.116636,0.382290,-0.378235,-0.667126,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
59,CC1(C)CCCC(C)(C)N1Cl,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,1.0,0.955033,-1.264246,0.621733,-0.341996,0.374763,-0.332263,-0.804000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
60,CN1CCCCC1,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,1.0,1.389185,-0.935876,0.111029,0.485331,0.308117,-0.724181,0.398491,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
61,CN1CCCCC1,[Li+].F[P-](F)(F)(F)(F)F,1.0,1.389185,-0.935876,0.111029,0.485331,0.308117,-0.724181,0.398491,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
62,CN1CCCCC1,[Li+].O=C1O[B-](F)(F)OC1=O,1.0,1.389185,-0.935876,0.111029,0.485331,0.308117,-0.724181,0.398491,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
63,CCN1CCCC1,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,1.0,0.876502,-1.072168,-0.081963,0.736596,0.206000,-0.486059,0.263508,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
64,CCN1CCCC1,[Li+].F[P-](F)(F)(F)(F)F,1.0,0.876502,-1.072168,-0.081963,0.736596,0.206000,-0.486059,0.263508,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
65,CCN1CCCC1,[Li+].O=C1O[B-](F)(F)OC1=O,1.0,0.876502,-1.072168,-0.081963,0.736596,0.206000,-0.486059,0.263508,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
66,CC1(C(=O)Cl)CCCCC1,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,1.0,0.578054,-0.216590,0.983576,-0.309683,0.916454,-0.045955,-1.378522,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
67,CC1(C(=O)Cl)CCCCC1,[Li+].F[P-](F)(F)(F)(F)F,1.0,0.578054,-0.216590,0.983576,-0.309683,0.916454,-0.045955,-1.378522,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [16]:
df_b1['solv_comb_sm'] = df_b1['solv_comb_sm'].apply(lambda x: Chem.MolToSmiles(Chem.MolFromSmiles(x)))
df_b1

/var/folders/s5/lmr5dh0n4bsgnpxrzwlts5lw0000gn/T/ipykernel_81833/4283590449.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_b1['solv_comb_sm'] = df_b1['solv_comb_sm'].apply(lambda x: Chem.MolToSmiles(Chem.MolFromSmiles(x)))


,solv_comb_sm,salt_comb_sm,batch,solv_ecfp_pca_0,solv_ecfp_pca_1,solv_ecfp_pca_2,solv_ecfp_pca_3,solv_ecfp_pca_4,solv_ecfp_pca_5,solv_ecfp_pca_6,...,norm_capacity_15,norm_capacity_16,norm_capacity_17,norm_capacity_18,norm_capacity_19,norm_capacity_20,norm_capacity_21,norm_capacity_22,norm_capacity_23,expt_test
58,CN1C(C)(C)CCCC1(C)C,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,1.0,1.154581,-1.168030,0.649902,-0.116636,0.382290,-0.378235,-0.667126,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
59,CC1(C)CCCC(C)(C)N1Cl,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,1.0,0.955033,-1.264246,0.621733,-0.341996,0.374763,-0.332263,-0.804000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
60,CN1CCCCC1,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,1.0,1.389185,-0.935876,0.111029,0.485331,0.308117,-0.724181,0.398491,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
61,CN1CCCCC1,[Li+].F[P-](F)(F)(F)(F)F,1.0,1.389185,-0.935876,0.111029,0.485331,0.308117,-0.724181,0.398491,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
62,CN1CCCCC1,[Li+].O=C1O[B-](F)(F)OC1=O,1.0,1.389185,-0.935876,0.111029,0.485331,0.308117,-0.724181,0.398491,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
63,CCN1CCCC1,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,1.0,0.876502,-1.072168,-0.081963,0.736596,0.206000,-0.486059,0.263508,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
64,CCN1CCCC1,[Li+].F[P-](F)(F)(F)(F)F,1.0,0.876502,-1.072168,-0.081963,0.736596,0.206000,-0.486059,0.263508,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
65,CCN1CCCC1,[Li+].O=C1O[B-](F)(F)OC1=O,1.0,0.876502,-1.072168,-0.081963,0.736596,0.206000,-0.486059,0.263508,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
66,CC1(C(=O)Cl)CCCCC1,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,1.0,0.578054,-0.216590,0.983576,-0.309683,0.916454,-0.045955,-1.378522,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
67,CC1(C(=O)Cl)CCCCC1,[Li+].F[P-](F)(F)(F)(F)F,1.0,0.578054,-0.216590,0.983576,-0.309683,0.916454,-0.045955,-1.378522,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [ ]:
miss_sm_b1 = 'CN1CCC(F)(F)CC1'

In [34]:
uniq_sm_b1 = df_b1['solv_comb_sm'].unique() ## miss_sm_b1 should also be present, so in total 15 unique molecules
uniq_sm_b1 = pd.DataFrame(uniq_sm_b1, columns=['solv_comb_sm'])
uniq_sm_b1

,solv_comb_sm
0,CN1C(C)(C)CCCC1(C)C
1,CC1(C)CCCC(C)(C)N1Cl
2,CN1CCCCC1
3,CCN1CCCC1
4,CC1(C(=O)Cl)CCCCC1
5,CC1(C)CCCC(=O)C1
6,O=C1CC(=O)CC2(CCCC2)C1
7,COC(=O)C1(C)CCCC1
8,FC1(F)CCCCC1Cl
9,FC1(F)CCCCC1


In [40]:
df_comm = df_unlabel_uniq.merge(uniq_sm_b1, on='solv_comb_sm', how='right') ## miss_sm_b1 should also be present, so in total 11 unique molecules
df_comm.dropna(inplace=True)
df_comm

,solv_comb_sm,salt_comb_sm,solv_ecfp_pca_0,solv_ecfp_pca_1,solv_ecfp_pca_2,solv_ecfp_pca_3,solv_ecfp_pca_4,solv_ecfp_pca_5,solv_ecfp_pca_6,solv_ecfp_pca_7,...,EI_1,EI_2,EI_3,EI_4,prediction_aggr,uncertainty_aggr,explore_aggr,exploit_aggr,ratio_aggr,EI_aggr
0,CN1C(C)(C)CCCC1(C)C,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,1.154581,-1.168030,0.649902,-0.116636,0.382290,-0.378235,-0.667126,1.046499,...,0.121238,0.233647,0.019349,0.314543,0.000003,0.001856,9.939627e-07,-2.116058e-07,-0.212891,7.823569e-07
1,CC1(C)CCCC(C)(C)N1Cl,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,0.955033,-1.264246,0.621733,-0.341996,0.374763,-0.332263,-0.804000,0.733635,...,0.117348,0.238667,0.019350,0.318957,0.000003,0.001891,9.990592e-07,-1.808677e-07,-0.181038,8.181915e-07
3,CCN1CCCC1,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,0.876502,-1.072168,-0.081963,0.736596,0.206000,-0.486059,0.263508,0.370727,...,0.048298,0.225977,0.017732,0.365997,0.000003,0.001920,9.487996e-07,-1.291486e-07,-0.136118,8.196509e-07
5,CC1(C)CCCC(=O)C1,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,0.544321,-0.787394,0.681496,-0.289700,0.458624,0.255901,-0.920301,0.736245,...,0.065880,0.198112,0.017264,0.277403,0.000003,0.001814,9.665304e-07,-2.673888e-07,-0.276648,6.991416e-07
8,FC1(F)CCCCC1Cl,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,1.447440,-1.230973,0.492616,-0.724459,0.302936,0.289000,-0.715427,-0.300911,...,0.107226,0.226766,0.019499,0.300796,0.000003,0.001854,9.931865e-07,-2.208576e-07,-0.222373,7.723289e-07
9,FC1(F)CCCCC1,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,0.680281,-1.383992,0.402121,-0.769924,0.378534,-0.445670,-0.916712,0.326252,...,0.171056,0.274104,0.021390,0.353475,0.000003,0.002002,1.004646e-06,-2.778286e-08,-0.027654,9.768629e-07
10,FC1(F)CCCCCC1,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,0.680281,-1.383992,0.402121,-0.769924,0.378534,-0.445670,-0.916712,0.326252,...,0.168578,0.267983,0.021286,0.344117,0.000003,0.001989,1.004117e-06,-4.619256e-08,-0.046003,9.579247e-07
11,CC1(C)CCCCC1,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,1.006541,-1.325178,0.425723,-0.443286,0.298219,0.452950,-0.998666,0.231133,...,0.146213,0.260781,0.020713,0.353354,0.000003,0.001941,1.005551e-06,-1.119255e-07,-0.111308,8.936253e-07
12,CC1(Cl)CCCCC1,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,0.737807,-1.412186,0.505232,-0.561888,0.444367,0.068359,-1.010924,0.569775,...,0.158994,0.266373,0.020798,0.354566,0.000003,0.001969,1.005323e-06,-7.406493e-08,-0.073673,9.312583e-07
13,ClC1(Cl)CCCC1,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,0.696267,-1.429634,0.575233,-0.595873,0.518935,-0.141128,-0.932757,0.463879,...,0.109057,0.249368,0.019436,0.331830,0.000003,0.001952,1.004380e-06,-1.240600e-07,-0.123519,8.803197e-07


In [ ]:
df_comm.to_csv('../../datasets/batch-1/labeled_batch1_uniq_solvents_wo_nmc_data.csv', index=False)

In [17]:
df_comm_label = df_unlabel_uniq.merge(df_b1, on=['solv_comb_sm'], how='left', suffixes=('', '_drop'))
df_comm_label.dropna(inplace=True)
df_comm_label = df_comm_label.loc[:, ~df_comm_label.columns.str.endswith('_drop')]
df_comm_label

,solv_comb_sm,salt_comb_sm,prediction_aggr,uncertainty_aggr,explore_aggr,exploit_aggr,ratio_aggr,EI_aggr,batch,solv_ecfp_pca_0,...,norm_capacity_15,norm_capacity_16,norm_capacity_17,norm_capacity_18,norm_capacity_19,norm_capacity_20,norm_capacity_21,norm_capacity_22,norm_capacity_23,expt_test
201,FC1(F)CCCCC1,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,0.000003,0.002002,1.004646e-06,-2.778286e-08,-0.027654,9.768629e-07,1.0,0.680281,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
202,FC1(F)CCCCC1,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,0.000003,0.002002,1.004646e-06,-2.778286e-08,-0.027654,9.768629e-07,1.0,0.680281,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
203,FC1(F)CCCCC1,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,0.000003,0.002002,1.004646e-06,-2.778286e-08,-0.027654,9.768629e-07,1.0,0.680281,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
246,FC1(F)CCCCCC1,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,0.000003,0.001989,1.004117e-06,-4.619256e-08,-0.046003,9.579247e-07,1.0,0.680281,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
247,FC1(F)CCCCCC1,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,0.000003,0.001989,1.004117e-06,-4.619256e-08,-0.046003,9.579247e-07,1.0,0.680281,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
248,FC1(F)CCCCCC1,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,0.000003,0.001989,1.004117e-06,-4.619256e-08,-0.046003,9.579247e-07,1.0,0.680281,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
338,CC1(Cl)CCCCC1,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,0.000003,0.001969,1.005323e-06,-7.406493e-08,-0.073673,9.312583e-07,1.0,0.737807,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
339,CC1(Cl)CCCCC1,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,0.000003,0.001969,1.005323e-06,-7.406493e-08,-0.073673,9.312583e-07,1.0,0.737807,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
340,CC1(Cl)CCCCC1,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,0.000003,0.001969,1.005323e-06,-7.406493e-08,-0.073673,9.312583e-07,1.0,0.737807,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
479,CC1(C)CCCCC1,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,0.000003,0.001941,1.005551e-06,-1.119255e-07,-0.111308,8.936253e-07,1.0,1.006541,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [ ]:
df_comm_label.to_csv('../../datasets/batch-1/labeled_batch1_all_wo_nmc_data.csv', index=False)